In [1]:
import numpy as np
import pandas as pd

import pickle as pkl

import gc
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from cuml.metrics import mean_absolute_error, mean_squared_error

import seaborn as sns

In [2]:
out_base_path = '/media/hiroki/working/kaggle/trends-neuroimaging/models'

In [3]:
OUTPUT_DICT = ''

ID = 'Id'
TARGET_COLS = ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']
SEED = 2020

N_FOLD = 15

In [4]:
#base_path2 = '/media/hiroki/share/kaggle_data/trends-assessment-prediction/tf_data'
#fnc = pd.read_csv(base_path2+'/pca'+'/fnc_pca.csv', dtype={'Id':str})
#loading = pd.read_csv(base_path2+'/ica'+'/loading_ica.csv', dtype={'Id':str})

In [5]:
base_path = '/media/hiroki/share/kaggle_data/trends-assessment-prediction/'
train = pd.read_csv(base_path+'train_scores.csv', dtype={'Id':str})\
            #.dropna().reset_index(drop=True) # to make things easy
reveal_ID = pd.read_csv(base_path+'reveal_ID_site2.csv', dtype={'Id':str})
ICN_numbers = pd.read_csv(base_path+'ICN_numbers.csv')
loading = pd.read_csv(base_path+'loading.csv', dtype={'Id':str})
fnc = pd.read_csv(base_path+'fnc.csv', dtype={'Id':str})

sample_submission = pd.read_csv(base_path+'sample_submission.csv', dtype={'Id':str})

In [6]:
site_base_path = '/media/hiroki/working/kaggle/trends-neuroimaging/site_label/adversal/'
site_train = pd.read_csv(site_base_path+'/train_prob_site2_icapca.csv', dtype={'Id':str})
site_test = pd.read_csv(site_base_path+'/test_prob_site2_icapca.csv', dtype={'Id':str})
site_train.drop(["is_site2", "is_site2_true"], axis=1, inplace=True)
site_test.drop(["is_site2", "is_site2_true"], axis=1, inplace=True)

In [7]:
site_train['prob_site2'] = zscore(site_train['prob_site2'], axis=0)
site_test['prob_site2'] = zscore(site_test['prob_site2'], axis=0)

In [8]:
fnc_features, loading_features = list(fnc.columns[1:]), list(loading.columns[1:])

In [9]:
sample_submission['ID_num'] = sample_submission[ID].apply(lambda x: int(x.split('_')[0]))
test = pd.DataFrame({ID: sample_submission['ID_num'].unique().astype(str)})
del sample_submission['ID_num']; gc.collect()
test.head()

,Id
0,10003
1,10006
2,10010
3,10011
4,10012


In [10]:
test.head()

,Id
0,10003
1,10006
2,10010
3,10011
4,10012


In [11]:
# merge
train = train.merge(loading, on=ID, how='left')
train = train.merge(fnc, on=ID, how='left')
train = train.merge(site_train, on=ID, how='left')
train

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2,IC_01,IC_07,IC_05,IC_16,...,CBN(18)_vs_DMN(94),CBN(4)_vs_DMN(94),CBN(7)_vs_DMN(94),CBN(18)_vs_CBN(13),CBN(4)_vs_CBN(13),CBN(7)_vs_CBN(13),CBN(4)_vs_CBN(18),CBN(7)_vs_CBN(18),CBN(7)_vs_CBN(4),prob_site2
0,10001,57.436077,30.571975,62.553736,53.325130,51.427998,0.006070,0.014466,0.004136,0.000658,...,0.552841,0.131046,0.335446,0.394867,-0.042853,0.124627,-0.060712,0.515964,0.290488,-1.541110
1,10002,59.580851,50.969456,67.470628,60.651856,58.311361,0.009087,0.009291,0.007049,-0.002076,...,-0.039792,0.143014,-0.189962,0.498373,0.444231,0.592438,0.028649,0.705524,0.248327,-1.248727
2,10004,71.413018,53.152498,58.012103,52.418389,62.536641,0.004675,0.000957,0.006154,-0.000429,...,0.309540,0.141469,0.030853,0.344394,0.214097,0.317556,0.012435,0.665937,0.081358,-0.150006
3,10005,66.532630,NaN,NaN,52.108977,69.993075,-0.000398,0.006878,0.009051,0.000369,...,0.394932,0.040443,0.428334,0.498837,0.266755,0.227379,0.028984,0.752343,0.087898,-0.227412
4,10007,38.617381,49.197021,65.674285,40.151376,34.096421,0.005192,0.010585,0.012160,-0.000920,...,0.408926,0.072004,0.157582,0.532046,0.355448,0.462675,0.161005,0.703679,0.293607,-0.329418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5872,21746,14.257265,21.358872,61.165998,51.778483,54.640179,-0.001115,0.007108,0.008652,0.003596,...,0.205351,0.012067,0.310750,0.426335,0.193812,0.158720,0.055471,0.568766,0.160516,-1.348176
5873,21747,55.456978,68.169675,29.907995,55.349257,54.019517,0.007263,0.016489,0.012704,0.004357,...,0.201846,-0.008290,0.119828,0.551936,0.598931,0.511816,0.303312,0.704483,0.461588,-1.178194
5874,21750,48.948756,55.114811,60.878271,38.617246,50.679885,0.005996,0.003873,0.012353,0.000242,...,0.375065,0.104857,0.262614,0.502715,0.322353,0.458041,0.343754,0.705207,0.341224,-0.151140
5875,21752,66.532630,59.844808,72.303110,55.458281,46.870235,0.000627,0.011407,0.010957,0.000534,...,0.431489,0.039062,0.119474,0.523894,0.445209,0.332011,0.228977,0.560968,0.263504,-0.339535


In [12]:
# merge
test = test.merge(loading, on=ID, how='left')
test = test.merge(fnc, on=ID, how='left')
test = test.merge(site_test, on=ID, how='left')
test

,Id,IC_01,IC_07,IC_05,IC_16,IC_26,IC_06,IC_10,IC_09,IC_18,...,CBN(18)_vs_DMN(94),CBN(4)_vs_DMN(94),CBN(7)_vs_DMN(94),CBN(18)_vs_CBN(13),CBN(4)_vs_CBN(13),CBN(7)_vs_CBN(13),CBN(4)_vs_CBN(18),CBN(7)_vs_CBN(18),CBN(7)_vs_CBN(4),prob_site2
0,10003,0.008151,0.014684,0.010444,-0.005293,-0.002913,0.015042,0.017745,0.003930,-0.008021,...,0.136850,-0.022361,0.137625,0.677972,0.409412,0.563892,0.438684,0.618204,0.284474,0.778386
1,10006,0.000334,0.005311,0.010053,0.006920,-0.000065,0.015310,0.016543,0.004794,0.003982,...,0.240957,0.270419,0.367692,0.354501,0.486364,0.416908,0.285274,0.693490,0.448526,-0.482380
2,10010,0.007103,0.006144,0.009770,-0.002884,-0.001346,0.015651,0.011613,-0.003291,0.013423,...,0.272077,0.193523,0.192254,0.563982,0.124482,0.488926,0.083368,0.774299,0.129327,-0.995876
3,10011,0.004362,0.010240,0.010167,0.004492,-0.001623,0.017381,0.014680,0.007453,0.008786,...,0.557121,0.042626,0.179456,0.416546,0.445402,0.436909,0.165182,0.591561,0.306678,1.261023
4,10012,-0.007521,-0.003918,0.008434,-0.001145,0.002017,0.015065,0.019616,0.004140,-0.003744,...,0.203298,0.173427,0.046047,0.561599,0.418268,0.609517,0.218285,0.790285,0.301010,3.367089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5872,21745,0.005406,0.006275,0.012252,0.003518,0.001400,0.015054,0.015373,0.001532,0.003546,...,0.580813,0.202241,0.254870,0.458581,0.434638,0.587167,0.009854,0.472956,0.342085,0.078809
5873,21748,0.004240,0.009213,0.010981,0.000443,-0.003072,0.010702,0.014673,0.005523,0.005780,...,0.234340,0.138985,0.343382,0.708744,0.312812,0.536501,0.214803,0.849512,0.204741,0.027008
5874,21749,0.004783,0.017910,0.012128,-0.005683,-0.011613,0.017000,0.007230,0.001315,0.008788,...,0.170154,-0.029638,0.383761,0.398305,0.578621,0.357127,0.009479,0.609545,0.317230,2.239513
5875,21751,0.003835,0.015067,0.015428,-0.002030,0.001205,0.012396,0.011026,-0.001491,0.005310,...,0.131902,-0.047932,0.022317,0.583869,0.596734,0.515209,0.379589,0.568422,0.439016,-0.573374


In [13]:
import IPython

def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

In [14]:
def metric(y_true, y_pred):
    return np.mean(np.sum(np.abs(y_true - y_pred), axis=0)/np.sum(y_true, axis=0))

In [15]:
# Giving less importance to FNC features since they are easier to overfit due to high dimensionality.
FNC_SCALE = 1/600

train[fnc_features] *= FNC_SCALE
test[fnc_features] *= FNC_SCALE

In [16]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(train_x, train_y, test_x, model, target_name):
    preds = []
    preds_test = []
    va_idxes = []
    
    score = []
    mae = []
    rmse = []

    # shuffleしなくても良い
    kf = KFold(n_splits=N_FOLD, shuffle=True, random_state=SEED)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx].values, train_x.iloc[va_idx].values
        tr_y, va_y = train_y.iloc[tr_idx].values, train_y.iloc[va_idx].values
        model.fit(tr_x, tr_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)
        
        score.append(metric(va_y, pred))
        mae.append(mean_absolute_error(va_y, pred))
        rmse.append(np.sqrt(mean_squared_error(va_y, pred)))
        
    score_cv = np.array(score).mean()
    mae_cv = np.array(mae).mean()
    rmse_cv = np.array(rmse).mean()
    print("{0}_score:{1}".format(target_name, np.round(score_cv, 8)))
    print("{0}_mae:{1}".format(target_name, np.array(mae_cv).mean()))
    print("{0}_rmse:{1}".format(target_name, np.array(rmse_cv).mean()))
    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test, score_cv

# 1st layer

## SVR

In [20]:
import json
f_selec_path = "/media/hiroki/working/kaggle/trends-neuroimaging/feature_selection"

json_path = f_selec_path+"/ridge_No2_columns.json"
with open(json_path) as f:
    f_selec = json.load(f)

In [21]:
f_keys = list(f_selec.keys())

In [22]:
import cudf
import cupy as cp
from cuml import SVR

In [23]:
gc.collect()

80

In [22]:
pred_train_targets_svr = {}
pred_test_targets_svr = {}

pred_train_targets_svr_dfs = {}
pred_test_targets_svr_dfs = {}

overal_score = 0

print("N_FOLD:{}".format(N_FOLD))

for target, c, feature, w in [("age", 80, f_selec[f_keys[0]], 0.3),
                              ("domain1_var1", 23, f_selec[f_keys[1]], 0.175),
                              ("domain1_var2", 1, f_selec[f_keys[2]], 0.175),
                              ("domain2_var1", 8, f_selec[f_keys[3]], 0.175),
                              ("domain2_var2", 15, f_selec[f_keys[4]], 0.175)]:
    
    #feature.append('prob_site2')
    train_df = train[train[target].notnull()]
    test_df = test
    
    use_idx = train_df.index
    train_x = train_df.drop([ID]+TARGET_COLS, axis=1)
    train_x = train_x[feature]
    train_y = train_df[target]
    test_x = test_df.drop(ID, axis=1)
    test_x = test_x[feature]
    
    svr = SVR(C=c, cache_size=3000.0)
    
    print("-----{}-----".format(target))
    pred_train, preds_test, score_cv = predict_cv(train_x, train_y, test_x, svr, target)
    overal_score += w*score_cv
    pred_train_targets_svr[target] = pred_train
    pred_test_targets_svr[target] = preds_test
    
    pred_train_targets_svr_dfs[target] = pd.Series(pred_train, name="pre_svr_{}".format(target), index=use_idx)
    pred_train_targets_svr_dfs[target] = pd.merge(train['Id'],
                                                  pred_train_targets_svr_dfs[target],
                                                  left_index=True,
                                                  right_index=True)
    pred_test_targets_svr_dfs[target] = pd.Series(preds_test, name="pre_svr_{}".format(target))
    pred_test_targets_svr_dfs[target] = pd.concat([test['Id'], pred_test_targets_svr_dfs[target]], axis=1)
    
    
print('--------------------------------------------')
print("Overal score:", np.round(overal_score, 8))

N_FOLD:15
-----age-----
age_score:0.14342984
age_mae:7.175251689121478
age_rmse:9.108591268483652
-----domain1_var1-----
domain1_var1_score:0.15129463
domain1_var1_mae:7.788501406921614
domain1_var1_rmse:9.71464785361684
-----domain1_var2-----
domain1_var2_score:0.15091099
domain1_var2_mae:8.938745618825989
domain1_var2_rmse:11.39431049714262
-----domain2_var1-----
domain2_var1_score:0.1816464
domain2_var1_mae:8.59360354045588
domain2_var1_rmse:10.916280836161347
-----domain2_var2-----
domain2_var2_score:0.17598432
domain2_var2_mae:9.130979812553077
domain2_var2_rmse:11.723112557340178
--------------------------------------------
Overal score: 0.15850031


3:1510
-----age-----
age_score:0.14386585
age_mae:7.197805992588449
age_rmse:9.147440070169111
-----domain1_var1-----
domain1_var1_score:0.15117448
domain1_var1_mae:7.7821543363146235
domain1_var1_rmse:9.704146603836946
-----domain1_var2-----
domain1_var2_score:0.15150079
domain1_var2_mae:8.974771341877382
domain1_var2_rmse:11.44197948977398
-----domain2_var1-----
domain2_var1_score:0.18195558
domain2_var1_mae:8.608609438557298
domain2_var1_rmse:10.934885564728182
-----domain2_var2-----
domain2_var2_score:0.17589358
domain2_var2_mae:9.128640415626789
domain2_var2_rmse:11.735250108597658
--------------------------------------------
Overal score: 0.15875153

In [23]:
for df in pred_train_targets_svr_dfs.values():
    display(df.describe())

,pre_svr_age
count,5877.000000
mean,50.188476
std,9.885064
min,20.331879
25%,42.962369


,pre_svr_domain1_var1
count,5439.000000
mean,51.704115
std,3.321249
min,41.108115
25%,49.482678


,pre_svr_domain1_var2
count,5439.000000
mean,60.071318
std,0.722236
min,57.495981
25%,59.586954


,pre_svr_domain2_var1
count,5838.000000
mean,47.901220
std,2.681473
min,38.960542
25%,46.022360


,pre_svr_domain2_var2
count,5838.000000
mean,52.668531
std,2.427612
min,44.416930
25%,51.020451


In [24]:
for df in pred_train_targets_svr_dfs.values():
    display(df)

,Id,pre_svr_age
0,10001,61.828573
1,10002,55.908563
2,10004,51.173828
3,10005,60.414452
4,10007,44.249488


,Id,pre_svr_domain1_var1
0,10001,53.896751
1,10002,55.602436
2,10004,51.527469
4,10007,51.104330
5,10008,48.931737


,Id,pre_svr_domain1_var2
0,10001,59.832226
1,10002,59.796948
2,10004,59.619253
4,10007,59.759595
5,10008,61.571164


,Id,pre_svr_domain2_var1
0,10001,48.922154
1,10002,49.377000
2,10004,48.468833
3,10005,50.201503
4,10007,46.510314


,Id,pre_svr_domain2_var2
0,10001,50.533125
1,10002,51.844046
2,10004,49.827237
3,10005,53.871136
4,10007,52.285553


In [25]:
pred_train_targets_svr_dfs

{'age':          Id  pre_svr_age
 0     10001    61.828573
 1     10002    55.908563
 2     10004    51.173828
 3     10005    60.414452
 4     10007    44.249488
 ...     ...          ...
 5872  21746    32.259654
 5873  21747    49.360546
 5874  21750    38.915942
 5875  21752    55.922405
 5876  21754    70.028159
 
 [5877 rows x 2 columns],
 'domain1_var1':          Id  pre_svr_domain1_var1
 0     10001             53.896751
 1     10002             55.602436
 2     10004             51.527469
 4     10007             51.104330
 5     10008             48.931737
 ...     ...                   ...
 5872  21746             50.418937
 5873  21747             48.560775
 5874  21750             48.562165
 5875  21752             54.381110
 5876  21754             59.010680
 
 [5439 rows x 2 columns],
 'domain1_var2':          Id  pre_svr_domain1_var2
 0     10001             59.832226
 1     10002             59.796948
 2     10004             59.619253
 4     10007             59.75959

In [27]:
with open (out_base_path+'/svm_train.pkl', 'wb') as f:
  pkl.dump(pred_train_targets_svr_dfs , f)

In [28]:
with open (out_base_path+'/svm_test.pkl', 'wb') as f:
  pkl.dump(pred_test_targets_svr_dfs , f)

## baysian ridge

In [16]:
from sklearn.linear_model import BayesianRidge

In [17]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [18]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(train_x, train_y, test_x, model, target_name):
    preds = []
    preds_test = []
    va_idxes = []
    
    score = []
    mae = []
    rmse = []

    # shuffleしなくても良い
    kf = KFold(n_splits=N_FOLD, shuffle=True, random_state=SEED)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx].values, train_x.iloc[va_idx].values
        tr_y, va_y = train_y.iloc[tr_idx].values, train_y.iloc[va_idx].values
        
        #tr_x = sc.fit_transform(tr_x)
        #va_x = sc.transform(va_x)
        #test_x = sc.transform(test_x)
        
        model.fit(tr_x, tr_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)
        
        score.append(metric(va_y, pred))
        mae.append(mean_absolute_error(va_y, pred))
        rmse.append(np.sqrt(mean_squared_error(va_y, pred)))
        
    score_cv = np.array(score).mean()
    mae_cv = np.array(mae).mean()
    rmse_cv = np.array(rmse).mean()
    print("{0}_score:{1}".format(target_name, np.round(score_cv, 8)))
    print("{0}_mae:{1}".format(target_name, np.array(mae_cv).mean()))
    print("{0}_rmse:{1}".format(target_name, np.array(rmse_cv).mean()))
    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test, score_cv

In [24]:
pred_train_targets_bayes_ridge = {}
pred_test_targets_bayes_ridge = {}

pred_train_targets_bayes_ridge_dfs = {}
pred_test_targets_bayes_ridge_dfs = {}

overal_score = 0

print("N_FOLD:{}".format(N_FOLD))

#for target, w in [("age", 0.3),
#                  ("domain1_var1", 0.175),
#                  ("domain1_var2", 0.175),
#                  ("domain2_var1", 0.175),
#                  ("domain2_var2", 0.175)]:
for target, feature, w in [("age", f_selec[f_keys[0]], 0.3),
                              ("domain1_var1", f_selec[f_keys[1]], 0.175),
                              ("domain1_var2", f_selec[f_keys[2]], 0.175),
                              ("domain2_var1", f_selec[f_keys[3]], 0.175),
                              ("domain2_var2", f_selec[f_keys[4]], 0.175)]:
    
    #feature.append("prob_site2")
    train_df = train[train[target].notnull()]
    test_df = test
    
    use_idx = train_df.index
    train_x = train_df.drop([ID]+TARGET_COLS, axis=1)
    train_x = train_x[feature]
    train_y = train_df[target]
    test_x = test_df.drop(ID, axis=1)
    test_x = test_x[feature]
    bayes_ridge = BayesianRidge(n_iter = 3000)
    
    print("-----{}-----".format(target))
    pred_train, preds_test, score_cv = predict_cv(train_x, train_y, test_x, bayes_ridge, target)
    overal_score += w*score_cv
    pred_train_targets_bayes_ridge[target] = pred_train
    pred_test_targets_bayes_ridge[target] = preds_test
    
    pred_train_targets_bayes_ridge_dfs[target] = pd.Series(pred_train, name="pre_bayRidge_selec_{}".format(target), index=use_idx)
    pred_train_targets_bayes_ridge_dfs[target] = pd.merge(train['Id'],
                                                 pred_train_targets_bayes_ridge_dfs[target],
                                                 left_index=True,
                                                 right_index=True)
    pred_test_targets_bayes_ridge_dfs[target] = pd.Series(preds_test, name="pre_bayRidge_selec_{}".format(target))
    pred_test_targets_bayes_ridge_dfs[target] = pd.concat([test['Id'], pred_test_targets_bayes_ridge_dfs[target]], axis=1)
print('--------------------------------------------')
print("Overal score:", np.round(overal_score, 8))

N_FOLD:15
-----age-----
age_score:0.1428682
age_mae:7.147244189856664
age_rmse:9.061260824814058
-----domain1_var1-----
domain1_var1_score:0.15089296
domain1_var1_mae:7.7680943144569135
domain1_var1_rmse:9.664221825956153
-----domain1_var2-----
domain1_var2_score:0.1511794
domain1_var2_mae:8.954957899510019
domain1_var2_rmse:11.363800644122856
-----domain2_var1-----
domain2_var1_score:0.18174716
domain2_var1_mae:8.598819580197647
domain2_var1_rmse:10.879469089247369
-----domain2_var2-----
domain2_var2_score:0.17644759
domain2_var2_mae:9.155206684876475
domain2_var2_rmse:11.68621704672863
--------------------------------------------
Overal score: 0.1584072


N_FOLD:15
-----age-----
age_score:0.14684742
age_mae:7.34615120925756
age_rmse:9.307663401862479
-----domain1_var1-----
domain1_var1_score:0.15121653
domain1_var1_mae:7.784724443181761
domain1_var1_rmse:9.68566087255278
-----domain1_var2-----
domain1_var2_score:0.15099406
domain1_var2_mae:8.944028892935641
domain1_var2_rmse:11.355687176417975
-----domain2_var1-----
domain2_var1_score:0.18179954
domain2_var1_mae:8.60136211616376
domain2_var1_rmse:10.880723091894087
-----domain2_var2-----
domain2_var2_score:0.17641739
domain2_var2_mae:9.153687256718134
domain2_var2_rmse:11.6864118703265
--------------------------------------------
Overal score: 0.15962904

In [ ]:
for df in pred_train_targets_bayes_ridge_dfs.values():
    display(df)

In [ ]:
for df in pred_train_targets_bayes_ridge_dfs.values():
    display(df)

In [ ]:
for df in pred_train_targets_bayes_ridge_dfs.values():
    display(df)

In [ ]:
pred_test_targets_df = pd.DataFrame(pred_test_targets_bayes_ridge)

In [ ]:
pred_test_targets_df.describe()

In [26]:
with open (out_base_path+'/1st_layer/bayes_ridge_pc_ica/bayes_ridge_selec_train.pkl', 'wb') as f:
  pkl.dump(pred_train_targets_bayes_ridge_dfs , f)

with open (out_base_path+'/1st_layer/bayes_ridge_pc_ica/bayes_ridge_selec_test.pkl', 'wb') as f:
  pkl.dump(pred_test_targets_bayes_ridge_dfs , f)

## BaggingRegressor

In [34]:
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import Ridge

In [35]:
pred_train_targets_bagging_ridge = {}
pred_test_targets_bagging_ridge = {}

pred_train_targets_bagging_ridge_dfs = {}
pred_test_targets_bagging_ridge_dfs = {}

overal_score = 0

print("N_FOLD:{}".format(N_FOLD))

for target, feature, w in [("age", f_selec[f_keys[0]], 0.3),
                              ("domain1_var1", f_selec[f_keys[1]], 0.175),
                              ("domain1_var2", f_selec[f_keys[2]], 0.175),
                              ("domain2_var1", f_selec[f_keys[3]], 0.175),
                              ("domain2_var2", f_selec[f_keys[4]], 0.175)]:
    
    train_df = train[train[target].notnull()]
    test_df = test
    
    use_idx = train_df.index
    train_x = train_df.drop([ID]+TARGET_COLS, axis=1)
    train_x = train_x[feature]
    train_y = train_df[target]
    test_x = test_df.drop(ID, axis=1)
    test_x = test_df[feature]
    bagging_ridge = BaggingRegressor(Ridge(alpha = 0.0001), n_estimators=15, random_state=42, max_samples=0.3, max_features=0.3)
    
    print("-----{}-----".format(target))
    pred_train, preds_test, score_cv = predict_cv(train_x, train_y, test_x, bagging_ridge, target)
    overal_score += w*score_cv
    pred_train_targets_bagging_ridge[target] = pred_train
    pred_test_targets_bagging_ridge[target] = preds_test
    
    pred_train_targets_bagging_ridge_dfs[target] = pd.Series(pred_train, name="pre_bagRidge_PCIC{}".format(target), index=use_idx)
    pred_train_targets_bagging_ridge_dfs[target] = pd.merge(train['Id'],
                                                 pred_train_targets_bagging_ridge_dfs[target],
                                                 left_index=True,
                                                 right_index=True)
    pred_test_targets_bagging_ridge_dfs[target] = pd.Series(preds_test, name="pre_bagRidge_PCIC{}".format(target))
    pred_test_targets_bagging_ridge_dfs[target] = pd.concat([test['Id'], pred_test_targets_bagging_ridge_dfs[target]], axis=1)
print('--------------------------------------------')
print("Overal score:", np.round(overal_score, 8))

N_FOLD:15
-----age-----
age_score:0.15629457
age_mae:7.818503656955662
age_rmse:9.898031385738998
-----domain1_var1-----
domain1_var1_score:0.15206823
domain1_var1_mae:7.828252666291275
domain1_var1_rmse:9.744513482535854
-----domain1_var2-----
domain1_var2_score:0.15116081
domain1_var2_mae:8.953944356641223
domain1_var2_rmse:11.367786915310251
-----domain2_var1-----
domain2_var1_score:0.18241217
domain2_var1_mae:8.630505210877539
domain2_var1_rmse:10.922195217752034
-----domain2_var2-----
domain2_var2_score:0.17633655
domain2_var2_mae:9.149655467693519
domain2_var2_rmse:11.672900241060184
--------------------------------------------
Overal score: 0.16273448


In [197]:
for df in pred_train_targets_bagging_ridge_dfs.values():
    display(df, head=False)

,Id,pre_bagRidge_PCICage
0,10001,54.314438
1,10002,53.500857
2,10004,49.053474
3,10005,54.372434
4,10007,46.516550
...,...,...
5872,21746,42.559280
5873,21747,51.670857
5874,21750,45.643191
5875,21752,51.841506


,Id,pre_bagRidge_PCICdomain1_var1
0,10001,53.343175
1,10002,53.224849
2,10004,51.317342
4,10007,50.309479
5,10008,49.548873
...,...,...
5872,21746,49.453572
5873,21747,49.356434
5874,21750,50.615331
5875,21752,53.416679


,Id,pre_bagRidge_PCICdomain1_var2
0,10001,58.971619
1,10002,57.672118
2,10004,59.532611
4,10007,58.768235
5,10008,59.683128
...,...,...
5872,21746,60.135169
5873,21747,59.907949
5874,21750,60.573529
5875,21752,60.336603


,Id,pre_bagRidge_PCICdomain2_var1
0,10001,48.217223
1,10002,46.395500
2,10004,45.831332
3,10005,47.471232
4,10007,46.767520
...,...,...
5872,21746,47.136502
5873,21747,48.334769
5874,21750,45.682052
5875,21752,46.899026


,Id,pre_bagRidge_PCICdomain2_var2
0,10001,50.099924
1,10002,52.195533
2,10004,50.559282
3,10005,51.677093
4,10007,52.826184
...,...,...
5872,21746,52.044722
5873,21747,53.353795
5874,21750,52.174772
5875,21752,51.660015


In [198]:
for df in pred_train_targets_bagging_ridge_dfs.values():
    display(df, head=False)

,Id,pre_bagRidge_PCICage
0,10001,54.314438
1,10002,53.500857
2,10004,49.053474
3,10005,54.372434
4,10007,46.516550
...,...,...
5872,21746,42.559280
5873,21747,51.670857
5874,21750,45.643191
5875,21752,51.841506


,Id,pre_bagRidge_PCICdomain1_var1
0,10001,53.343175
1,10002,53.224849
2,10004,51.317342
4,10007,50.309479
5,10008,49.548873
...,...,...
5872,21746,49.453572
5873,21747,49.356434
5874,21750,50.615331
5875,21752,53.416679


,Id,pre_bagRidge_PCICdomain1_var2
0,10001,58.971619
1,10002,57.672118
2,10004,59.532611
4,10007,58.768235
5,10008,59.683128
...,...,...
5872,21746,60.135169
5873,21747,59.907949
5874,21750,60.573529
5875,21752,60.336603


,Id,pre_bagRidge_PCICdomain2_var1
0,10001,48.217223
1,10002,46.395500
2,10004,45.831332
3,10005,47.471232
4,10007,46.767520
...,...,...
5872,21746,47.136502
5873,21747,48.334769
5874,21750,45.682052
5875,21752,46.899026


,Id,pre_bagRidge_PCICdomain2_var2
0,10001,50.099924
1,10002,52.195533
2,10004,50.559282
3,10005,51.677093
4,10007,52.826184
...,...,...
5872,21746,52.044722
5873,21747,53.353795
5874,21750,52.174772
5875,21752,51.660015


In [199]:
with open (out_base_path+'/1st_layer/bagging_ridge_pc_ica/bagging_ridge_train.pkl', 'wb') as f:
  pkl.dump(pred_train_targets_bagging_ridge_dfs , f)

with open (out_base_path+'/1st_layer/bagging_ridge_pc_ica/bagging_ridge_test.pkl', 'wb') as f:
  pkl.dump(pred_test_targets_bagging_ridge_dfs , f)

# 2nd layer

In [40]:
with open(out_base_path + '/1st_layer/svm/svm_train.pkl', 'rb') as f:
  pred_train_targets_svr_dfs = pkl.load(f)

with open(out_base_path + '/1st_layer/svm/svm_test.pkl', 'rb') as f:
  pred_test_targets_svr_dfs = pkl.load(f)

In [41]:
with open(out_base_path + '/1st_layer/bayes_ridge/bayes_ridge_train.pkl', 'rb') as f:
  pred_train_targets_bayes_ridge_dfs = pkl.load(f)

with open(out_base_path + '/1st_layer/bayes_ridge/bayes_ridge_test.pkl', 'rb') as f:
  pred_test_targets_bayes_ridge_dfs = pkl.load(f)

In [42]:
with open(out_base_path + '/1st_layer/bagging_ridge/bagging_ridge_train.pkl', 'rb') as f:
  pred_train_targets_bagging_ridge_dfs = pkl.load(f)

with open(out_base_path + '/1st_layer/bagging_ridge/bagging_ridge_test.pkl', 'rb') as f:
  pred_test_targets_bagging_ridge_dfs = pkl.load(f)

In [43]:
for i, pred_df in enumerate(pred_train_targets_svr_dfs.values()):
    if i == 0:
        pred_train_dfs = pred_df
        #display(pred_train_dfs, head=False)
    else:
        pred_train_dfs = pd.concat([pred_train_dfs, pred_df.drop("Id", axis=1)], axis=1)
        #display(pred_train_dfs, head=False)

for i, pred_df in enumerate(pred_train_targets_bayes_ridge_dfs.values()):
    pred_train_dfs = pd.concat([pred_train_dfs, pred_df.drop("Id", axis=1)], axis=1)

for i, pred_df in enumerate(pred_train_targets_bagging_ridge_dfs.values()):
    pred_train_dfs = pd.concat([pred_train_dfs, pred_df.drop("Id", axis=1)], axis=1)

pred_train_dfs = pd.concat([pred_train_dfs, train[TARGET_COLS]], axis=1)

In [44]:
for i, pred_df in enumerate(pred_test_targets_svr_dfs.values()):
    if i == 0:
        pred_test_dfs = pred_df
        #display(pred_test_dfs, head=False)
    else:
        pred_test_dfs = pd.concat([pred_test_dfs, pred_df.drop("Id", axis=1)], axis=1)
        #display(pred_test_dfs, head=False)

for i, pred_df in enumerate(pred_test_targets_bayes_ridge_dfs.values()):
    pred_test_dfs = pd.concat([pred_test_dfs, pred_df.drop("Id", axis=1)], axis=1)

for i, pred_df in enumerate(pred_test_targets_bagging_ridge_dfs.values()):
    pred_test_dfs = pd.concat([pred_test_dfs, pred_df.drop("Id", axis=1)], axis=1)

In [45]:
pred_train_dfs

,Id,pre_svr_age,pre_svr_domain1_var1,pre_svr_domain1_var2,pre_svr_domain2_var1,pre_svr_domain2_var2,pre_bayRidge_age,pre_bayRidge_domain1_var1,pre_bayRidge_domain1_var2,pre_bayRidge_domain2_var1,...,pre_bagRidge_age,pre_bagRidge_domain1_var1,pre_bagRidge_domain1_var2,pre_bagRidge_domain2_var1,pre_bagRidge_domain2_var2,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10001,60.877768,53.299463,59.728622,49.506853,50.245806,60.906731,53.632844,58.933031,48.193288,...,60.274152,54.624309,58.081457,49.031462,50.733896,57.436077,30.571975,62.553736,53.325130,51.427998
1,10002,55.056918,54.708045,59.462757,48.809679,52.669451,56.150469,53.087283,59.347237,47.758491,...,54.894260,53.095975,58.119624,44.822946,51.268865,59.580851,50.969456,67.470628,60.651856,58.311361
2,10004,51.727589,51.827130,60.177994,47.702372,50.088077,52.027515,52.085337,58.739435,46.646664,...,52.387315,51.403625,59.881164,47.289573,48.887204,71.413018,53.152498,58.012103,52.418389,62.536641
3,10005,61.276086,NaN,NaN,50.182354,54.036360,60.813771,NaN,NaN,48.631294,...,54.920694,NaN,NaN,46.401744,51.278491,66.532630,NaN,NaN,52.108977,69.993075
4,10007,44.009071,49.981281,58.200460,46.080720,52.361837,43.660350,49.480579,59.537922,46.166929,...,45.028738,49.663154,56.098351,46.403485,51.736203,38.617381,49.197021,65.674285,40.151376,34.096421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5872,21746,33.566653,50.700319,60.844463,45.499232,50.331900,34.199012,50.407656,59.896793,45.761590,...,40.155311,48.399123,59.641840,46.786267,49.793648,14.257265,21.358872,61.165998,51.778483,54.640179
5873,21747,51.158271,48.903705,58.519688,47.681367,54.147085,51.591765,48.533070,58.785206,48.017085,...,50.833635,48.752210,57.868394,47.029436,52.856192,55.456978,68.169675,29.907995,55.349257,54.019517
5874,21750,40.541940,49.380542,62.664363,46.724739,53.457566,41.323194,49.492569,59.906589,46.374749,...,45.212725,51.879631,61.189195,47.063534,51.826613,48.948756,55.114811,60.878271,38.617246,50.679885
5875,21752,55.981565,54.848841,58.994825,48.854849,54.597196,55.273854,55.548991,59.230211,48.076530,...,52.072927,55.656525,58.527484,46.580832,50.755409,66.532630,59.844808,72.303110,55.458281,46.870235


In [46]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [53]:
layer2_pred_train_targets_bayes_ridge = {}
layer2_pred_test_targets_bayes_ridge = {}

layer2_pred_train_targets_bayes_ridge_dfs = {}
layer2_pred_test_targets_bayes_ridge_dfs = {}

overal_score = 0

print("N_FOLD:{}".format(N_FOLD))

for target, w in [("age", 0.3),
                  ("domain1_var1", 0.175),
                  ("domain1_var2", 0.175),
                  ("domain2_var1", 0.175),
                  ("domain2_var2", 0.175)]:
    target_columns = pred_train_dfs.loc[
        :, pred_train_dfs.columns.str.contains('_'+target)
    ].columns
    train_df2 = pred_train_dfs[pred_train_dfs[target].notnull()]
    use_idx = train_df2.index
    train_x_2 = train_df2[target_columns]
    ss.fit(train_x_2)
    train_x_2 = ss.fit_transform(train_x_2)
    train_x_2 = pd.DataFrame(train_x_2)
    
    train_y = train_df2[target]
    
    test_df2 = pred_test_dfs
    test_x_2 = test_df2[target_columns]
    test_x_2 = ss.fit_transform(test_x_2)
    test_x_2 = pd.DataFrame(test_x_2)
    
    bayes_ridge = BayesianRidge(n_iter = 3000)
    
    print("-----{}-----".format(target))
    pred_train, preds_test, score_cv = predict_cv(train_x_2, train_y, test_x_2, bayes_ridge, target)
    overal_score += w*score_cv
    layer2_pred_train_targets_bayes_ridge[target] = pred_train
    layer2_pred_test_targets_bayes_ridge[target] = preds_test
    
    layer2_pred_train_targets_bayes_ridge_dfs[target] = pd.Series(pred_train, name="pre_bayRidge_2{}".format(target), index=use_idx)
    layer2_pred_train_targets_bayes_ridge_dfs[target] = pd.merge(train['Id'],
                                                 pred_train_targets_bayes_ridge_dfs[target],
                                                 left_index=True,
                                                 right_index=True)
    layer2_pred_test_targets_bayes_ridge_dfs[target] = pd.Series(preds_test, name="pre_bayRidge_2{}".format(target))
    layer2_pred_test_targets_bayes_ridge_dfs[target] = pd.concat([test['Id'], pred_test_targets_bayes_ridge_dfs[target]], axis=1)

    
    
print('--------------------------------------------')
print("Overal score:", np.round(overal_score, 8))

N_FOLD:5
-----age-----
age_score:0.14290134
age_mae:7.149529179204238
age_rmse:9.075374630095723
-----domain1_var1-----
domain1_var1_score:0.15088524
domain1_var1_mae:7.7673042896042315
domain1_var1_rmse:9.675497704182074
-----domain1_var2-----
domain1_var2_score:0.1510644
domain1_var2_mae:8.949167631465432
domain1_var2_rmse:11.363296479985333
-----domain2_var1-----
domain2_var1_score:0.18185343
domain2_var1_mae:8.603910642572405
domain2_var1_rmse:10.887120573571538
-----domain2_var2-----
domain2_var2_score:0.1761039
domain2_var2_mae:9.139730031829178
domain2_var2_rmse:11.67886425063955
--------------------------------------------
Overal score: 0.15835412


In [50]:
layer2_pred_train_targets_bayes_ridge

{'age': array([60.92924035, 55.73604921, 51.81919287, ..., 40.97878948,
        55.48744612, 69.10405092]),
 'domain1_var1': array([53.84935261, 53.62270235, 51.86451086, ..., 49.90640618,
        55.63106104, 58.50533207]),
 'domain1_var2': array([58.75069171, 58.89048743, 59.30978808, ..., 60.43484748,
        59.12801405, 59.48083869]),
 'domain2_var1': array([48.72933569, 46.66900097, 46.8495291 , ..., 46.58282476,
        47.3205666 , 50.67291197]),
 'domain2_var2': array([50.38281153, 51.63701494, 49.29237313, ..., 52.06460545,
        51.77865179, 53.18323709])}

In [66]:
for df in layer2_pred_train_targets_bayes_ridge_dfs.values():
    display(df.describe())
#pred_train_targets_df = pd.DataFrame(layer2_pred_train_targets_bayes_ridge)
pred_test_targets_df = pd.DataFrame(layer2_pred_test_targets_bayes_ridge)

,pre_bayRidge_age
count,5877.000000
mean,50.038717
std,10.038128
min,19.518356
25%,42.842211


,pre_bayRidge_domain1_var1
count,5439.000000
mean,51.492404
std,3.185482
min,40.796837
25%,49.327597


,pre_bayRidge_domain1_var2
count,5439.000000
mean,59.245596
std,0.645455
min,56.373948
25%,58.836050


,pre_bayRidge_domain2_var1
count,5838.000000
mean,47.321834
std,2.256074
min,40.439732
25%,45.751823


,pre_bayRidge_domain2_var2
count,5838.000000
mean,51.903249
std,1.865373
min,40.634094
25%,50.685836


In [58]:
with open (out_base_path+'/2nd_layer/bayes_ridge/bayes_ridge_train.pkl', 'wb') as f:
  pkl.dump(layer2_pred_test_targets_bayes_ridge_dfs , f)

with open (out_base_path+'/2nd_layer/bayes_ridge/bayes_ridge_test.pkl', 'wb') as f:
  pkl.dump(pred_test_targets_bagging_ridge_dfs , f)

# Submit

In [67]:
test_df = pd.concat([test['Id'], pred_test_targets_df], axis=1)

In [68]:
test_df

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10003,55.738271,49.272082,58.923767,48.935058,56.170406
1,10006,63.345616,54.309045,58.703233,48.824776,51.730516
2,10010,39.413663,49.760247,59.352942,45.258089,52.595287
3,10011,51.463856,52.165475,59.870948,48.533458,50.852033
4,10012,52.983837,55.607438,56.904215,47.033798,57.313630
...,...,...,...,...,...,...
5872,21745,40.615615,51.048883,58.895316,46.151702,53.875924
5873,21748,56.423210,56.542430,59.787313,47.663472,52.674281
5874,21749,41.860067,46.538843,59.483453,45.494613,50.876778
5875,21751,41.134915,50.540838,58.966287,45.408914,55.829795


In [69]:
test_df.describe()

,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
count,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000
mean,50.034353,51.474304,59.243701,47.325159,51.906001
std,10.056516,3.187426,0.761660,2.329861,1.954629
min,18.409352,40.724298,56.108591,39.214262,44.376592
25%,42.773848,49.283917,58.728511,45.703129,50.550420
50%,49.779117,51.506635,59.258790,47.325113,51.857983
75%,57.178739,53.672268,59.767707,48.975285,53.178788
max,82.650448,63.315376,61.618214,55.540099,60.278996


In [72]:
test_df.to_csv('/media/hiroki/working/kaggle/trends-neuroimaging/models/submit/stacking_pre_test_df.csv')

In [73]:
sub_df = pd.melt(test_df[["Id", "age", "domain1_var1", "domain1_var2", "domain2_var1", "domain2_var2"]], id_vars=["Id"], value_name="Predicted")
sub_df["Id"] = sub_df["Id"].astype("str") + "_" +  sub_df["variable"].astype("str")

sub_df = sub_df.drop("variable", axis=1).sort_values("Id")
assert sub_df.shape[0] == test_df.shape[0]*5
sub_df.head(10)

,Id,Predicted
0,10003_age,55.738271
5877,10003_domain1_var1,49.272082
11754,10003_domain1_var2,58.923767
17631,10003_domain2_var1,48.935058
23508,10003_domain2_var2,56.170406
1,10006_age,63.345616
5878,10006_domain1_var1,54.309045
11755,10006_domain1_var2,58.703233
17632,10006_domain2_var1,48.824776
23509,10006_domain2_var2,51.730516


In [74]:
sub_df.to_csv('/media/hiroki/working/kaggle/trends-neuroimaging/models/submit/submit.csv', index=False)